In [3]:
import sys

sys.path.append('/root/lib/')

import ROOT as r
import os
import json
import pandas as pd
import uproot 
import awkward as ak
import array as arr
import numpy as np
import sys

sys.path.append(os.getcwd() + '/../utilities/')
from milliqanProcessor import *
from milliqanScheduler import *
from milliqanCuts import *
from milliqanPlotter import *

%jsroot on
%load_ext autoreload
%autoreload 2

In [4]:
def loadJson(jsonFile):
    fin = open(jsonFile)
    data = json.load(fin)
    lumis = pd.DataFrame(data['data'], columns=data['columns'])
    return lumis

milliqanOfflinePath = '/home/mcarrigan/scratch0/milliQan/analysis/milliqanOffline/Run3Detector/'
goodRuns = loadJson(milliqanOfflinePath + '/configuration/barConfigs/goodRunsList.json')

goodRuns.loc[goodRuns['run'] == 1100]

,run,file,goodRunLoose,goodRunMedium,goodRunTight,goodSingleTrigger,tag
6412,1100,1,False,False,False,True,v1p0


In [5]:
milliqanOfflinePath = '/home/mcarrigan/scratch0/milliQan/analysis/milliqanOffline/Run3Detector/'
lumis = loadJson(milliqanOfflinePath + '/configuration/barConfigs/mqLumis.json')
lumis['file'] = lumis['file'].astype(int)
#lumis.loc[(lumis['lumiEst'] > 0) & (lumis['run'] > 1400)]
#test = lumis.loc[(lumis['run'] < 1200) & (lumis['run'] >= 1100)]
test = lumis.loc[(lumis['run'] >= 1100) & (lumis['run'] < 1300) & (lumis['beam'] == False)]

test

,run,file,lumis,fill,beam,dir,filename,start,stop,beamType,beamEnergy,betaStar,fillStart,fillEnd,startStableBeam,endStableBeam,lumiEst
31935,1212,296,0,None,False,/store/user/milliqan/run3/bar//1200/0001,MilliQan_Run1212.296_default.root,1698542757000,1698543190000,None,None,None,None,None,None,None,0.0
31936,1212,297,0,None,False,/store/user/milliqan/run3/bar//1200/0001,MilliQan_Run1212.297_default.root,1698543190000,1698543606000,None,None,None,None,None,None,None,0.0
31937,1212,298,0,None,False,/store/user/milliqan/run3/bar//1200/0001,MilliQan_Run1212.298_default.root,1698543606000,1698544048000,None,None,None,None,None,None,None,0.0
31938,1212,299,0,None,False,/store/user/milliqan/run3/bar//1200/0001,MilliQan_Run1212.299_default.root,1698544048000,1698544468000,None,None,None,None,None,None,None,0.0
31941,1212,300,0,None,False,/store/user/milliqan/run3/bar//1200/0001,MilliQan_Run1212.300_default.root,1698544469000,1698544922000,None,None,None,None,None,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34282,1296,1,0,None,False,/store/user/milliqan/run3/bar//1200/0009,MilliQan_Run1296.1_default.root,1699878555000,1699878675000,None,None,None,None,None,None,None,0.0
34283,1297,1,0,None,False,/store/user/milliqan/run3/bar//1200/0009,MilliQan_Run1297.1_default.root,1699878865000,1699878988000,None,None,None,None,None,None,None,0.0
34284,1298,1,0,None,False,/store/user/milliqan/run3/bar//1200/0009,MilliQan_Run1298.1_default.root,1699879167000,1699879425000,None,None,None,None,None,None,None,0.0
34285,1299,1,0,None,False,/store/user/milliqan/run3/bar//1200/0009,MilliQan_Run1299.1_default.root,1699879473000,1699879606000,None,None,None,None,None,None,None,0.0


In [6]:
checkRun = 1100
checkFile = 2
value = goodRuns.loc[(goodRuns['run'] == checkRun) & (goodRuns['file'] == checkFile)]

len(value)

0

In [27]:
#get list of files to look at
files = []

beam = True

dataDir = '/store/user/milliqan/trees/v34/1100/'

for ifile, filename in enumerate(os.listdir(dataDir)):
    #if len(files) > 10: break
    #if ifile > 10: break
    
    if not filename.endswith('root'): continue
    
    runNum = int(filename.split('Run')[1].split('.')[0])
    fileNum = int(filename.split('.')[1].split('_')[0])
        
    goodRun = goodRuns['goodRunTight'].loc[(goodRuns['run'] == runNum) & (goodRuns['file'] == fileNum)]
    beamOn = lumis['beam'].loc[(lumis['run'] == runNum) & (lumis['file'] == fileNum)]

    if len(goodRun) == 0 or len(beamOn) == 0: continue

    print(filename, runNum, fileNum, beamOn.item(), goodRun.item())
    
    if beam:
        if goodRun.item() and beamOn.item(): files.append(dataDir+filename)
    else:
        if goodRun.item() and not beamOn.item(): files.append(dataDir+filename)


files, len(files)

MilliQan_Run1100.1_v34.root 1100 1 True False
MilliQan_Run1101.1_v34.root 1101 1 True True
MilliQan_Run1101.2_v34.root 1101 2 True True
MilliQan_Run1101.3_v34.root 1101 3 True True
MilliQan_Run1101.4_v34.root 1101 4 True True
MilliQan_Run1102.2_v34.root 1102 2 True False
MilliQan_Run1103.10_v34.root 1103 10 True False
MilliQan_Run1103.11_v34.root 1103 11 True False
MilliQan_Run1103.13_v34.root 1103 13 True False
MilliQan_Run1103.14_v34.root 1103 14 True False
MilliQan_Run1103.15_v34.root 1103 15 True False
MilliQan_Run1103.16_v34.root 1103 16 True False
MilliQan_Run1103.17_v34.root 1103 17 True False
MilliQan_Run1103.18_v34.root 1103 18 True False
MilliQan_Run1103.19_v34.root 1103 19 True False
MilliQan_Run1103.20_v34.root 1103 20 True False
MilliQan_Run1103.21_v34.root 1103 21 True False
MilliQan_Run1103.2_v34.root 1103 2 True False
MilliQan_Run1103.3_v34.root 1103 3 True False
MilliQan_Run1103.5_v34.root 1103 5 True False
MilliQan_Run1103.6_v34.root 1103 6 True False
MilliQan_Run1103

(['/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.1_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.2_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.3_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1101.4_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1000_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1001_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1002_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1003_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1004_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1005_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1006_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1007_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1008_v34.root',
  '/store/user/milliqan/trees/v34/1100/MilliQan_Run1114.1009_v34.root',
  '/

In [8]:
@mqCut
def pulseTime(self):
    events = self.events
    straightPath = self.events['straightPulseCut']

    timeToUse = 'timeFit_module_calibrated'

    self.events['straightPathL0Time'] = events[timeToUse][events['layer0'] & events['straightPulseCut']]
    self.events['straightPathL1Time'] = events[timeToUse][events['layer1'] & events['straightPulseCut']]
    self.events['straightPathL2Time'] = events[timeToUse][events['layer2'] & events['straightPulseCut']]
    self.events['straightPathL3Time'] = events[timeToUse][events['layer3'] & events['straightPulseCut']]

    height0 = ak.max(self.events['height'][events['layer0'] & events['straightPulseCut']], axis=1)
    height1 = ak.max(self.events['height'][events['layer1'] & events['straightPulseCut']], axis=1)
    height2 = ak.max(self.events['height'][events['layer2'] & events['straightPulseCut']], axis=1)
    height3 = ak.max(self.events['height'][events['layer3'] & events['straightPulseCut']], axis=1)

    mask0 = (self.events['height'][events['layer0'] & events['straightPulseCut']] == height0)
    mask1 = (self.events['height'][events['layer1'] & events['straightPulseCut']] == height1)
    mask2 = (self.events['height'][events['layer2'] & events['straightPulseCut']] == height2)
    mask3 = (self.events['height'][events['layer3'] & events['straightPulseCut']] == height3)

    self.events['straightPathL0Time'] = self.events['straightPathL0Time'][mask0]
    self.events['straightPathL1Time'] = self.events['straightPathL1Time'][mask1]
    self.events['straightPathL2Time'] = self.events['straightPathL2Time'][mask2]
    self.events['straightPathL3Time'] = self.events['straightPathL3Time'][mask3]

    self.events['straightPathDiffL03'] = self.events['straightPathL3Time'] - self.events['straightPathL0Time']

    '''output = self.events['straightPathL0Time']
    drop_empty = ak.num(output) > 0
    print("output", ak.drop_none(output[drop_empty]))'''


In [9]:
@mqCut
def centralTime(self):
    events = self.events
    timeCut = (events['timeFit_module_calibrated'] > 1100) & (events['timeFit_module_calibrated'] < 1400)
    drop_empty = ak.num(timeCut) > 0
    #newEvents = events[timeCut]
    newEvents = drop_empty & timeCut
    #print()
    #self.events = events[newEvents]
    for branch in branches:
        #print(branch, type(branch))
        self.events[branch] = self.events[branch][newEvents]


In [10]:
'''@mqCut
def heightCutMod(self, cutName='heightCut', heightCut=800, cut=True, branches=None):
    self.events[cutName] = self.events['height'] >= int(heightCut)
    print(self.events[cutName])
    print(self.events['height'][self.events[cutName]])
    if cut:
        print("In cut", branches)
        for branch in branches:
            print("branch", branch)
            self.events[branch] = self.events[branch][self.events[cutName]]'''

'@mqCut\ndef heightCutMod(self, cutName=\'heightCut\', heightCut=800, cut=True, branches=None):\n    self.events[cutName] = self.events[\'height\'] >= int(heightCut)\n    print(self.events[cutName])\n    print(self.events[\'height\'][self.events[cutName]])\n    if cut:\n        print("In cut", branches)\n        for branch in branches:\n            print("branch", branch)\n            self.events[branch] = self.events[branch][self.events[cutName]]'

need to figure out how to select the max pulse in the time window only

In [36]:
#define a file list to run over
filelist = files[:2]

#define the necessary branches to run over
branches = ['event', 'tTrigger', 'boardsMatched', 'pickupFlag', 'fileNumber', 'runNumber', 
            'time_module_calibrated', 'timeFit_module_calibrated', 'row', 'column', 'layer', 'height', 'area']

#define the milliqan cuts object
mycuts = milliqanCuts()

#require pulses are not pickup
pickupCut = mycuts.getCut(mycuts.pickupCut, 'pickupCut', cut=True, branches=branches)

#require that all digitizer boards are matched
boardMatchCut = mycuts.getCut(mycuts.boardsMatched, 'boardMatchCut', cut=True, branches=branches)

#height cut to get large pulses
muonHeightCut = mycuts.getCut(mycuts.heightCut, 'muonHeightCut', heightCut=1200, cut=True, branches=branches)

#muon area cut
muonAreaCut = mycuts.getCut(mycuts.areaCut, 'muonAreaCut', areaCut=500000, cut=True, branches=branches)

#define milliqan plotter
myplotter = milliqanPlotter()

#create root histogram 
bins = 400
xmin = 1100
xmax = 1500

h_pulseTime03 = r.TH2F("h_pulseTime03", "Pulse Times Between Layer 0 and 3", bins, xmin, xmax, bins, xmin, xmax)
h_L0Times = r.TH1F('h_L0Times', "Pulse Times Layer 0", 400, 1100, 1500)
h_L3Times = r.TH1F('h_L3Times', "Pulse Times Layer 3", 400, 1100, 1500)
h_TimeDiff = r.TH1F('h_TimeDiff', "Difference in Layer 0 and 3 Times", 100, -50, 50)
h_height = r.TH1F('h_height', "Height of Passing Pulses", 650, 0, 1300)
h_area = r.TH1F('h_area', "Area of Passing Pulses", 1000, 0, 100e3)

setattr(milliqanCuts, "pulseTime", pulseTime)
setattr(milliqanCuts, "centralTime", centralTime)
setattr(milliqanCuts, "muonHeightCut", muonHeightCut)


#add root histogram to plotter
myplotter.addHistograms(h_pulseTime03, ['straightPathL0Time', 'straightPathL3Time'])
myplotter.addHistograms(h_L0Times, 'straightPathL0Time')
myplotter.addHistograms(h_L3Times, 'straightPathL3Time')
myplotter.addHistograms(h_TimeDiff, 'straightPathDiffL03')
myplotter.addHistograms(h_height, 'height')
myplotter.addHistograms(h_area, 'area')

#defining the cutflow
#cutflow = [boardMatchCut, pickupCut, 
cutflow = [boardMatchCut, pickupCut, mycuts.centralTime, muonAreaCut, mycuts.layerCut, mycuts.straightLineCut, 
            mycuts.pulseTime, myplotter.dict['h_pulseTime03'], myplotter.dict['h_height'], myplotter.dict['h_area'],
            myplotter.dict['h_L0Times'], myplotter.dict['h_L3Times'], myplotter.dict['h_TimeDiff']]

#create a schedule of the cuts
myschedule = milliQanScheduler(cutflow, mycuts, myplotter)

#print out the schedule
myschedule.printSchedule()

#create the milliqan processor object
myiterator = milliqanProcessor(filelist, branches, myschedule, step_size=10000)

#run the milliqan processor
myiterator.run()

#save plots
myplotter.saveHistograms("layerTimes_april29_debug.root")

<bound method mqCut.<locals>.inner1 of <milliqanCuts.milliqanCuts object at 0x7efd9d41bfa0>> <class 'method'>
<bound method milliqanCuts.layerCut of <milliqanCuts.milliqanCuts object at 0x7efd9d41bfa0>> <class 'method'>
<bound method milliqanCuts.straightLineCut of <milliqanCuts.milliqanCuts object at 0x7efd9d41bfa0>> <class 'method'>
<bound method mqCut.<locals>.inner1 of <milliqanCuts.milliqanCuts object at 0x7efd9d41bfa0>> <class 'method'>
----------------------------
MilliQan Scheduler:
	0. boardMatchCut
	1. pickupCut
	2. centralTime
	3. muonAreaCut
	4. layerCut
	5. straightLineCut
	6. pulseTime
	7. h_pulseTime03
	8. h_height
	9. h_area
	10. h_L0Times
	11. h_L3Times
	12. h_TimeDiff
----------------------------
MilliQan Processor: Processing event 0...
MilliQan Processor: Processing event 1000...
Number of processed events 2000


Warning in <TFile::Append>: Replacing existing TH1: h_pulseTime03 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_TimeDiff (Potential memory leak).


In [29]:
c1 = r.TCanvas("c1", "c1", 600, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [30]:
h_pulseTime03.SetMarkerSize(10)
h_pulseTime03.SetMarkerStyle(7)
h_pulseTime03.Draw("")
line0 = r.TLine(1150, 1150, 1450, 1450)
line40p = r.TLine(1190, 1150, 1490, 1450)
line40m = r.TLine(1110, 1150, 1410, 1450)
line0.Draw("same")
line40p.Draw("same")
line40m.Draw("same")
c1.Draw()

In [18]:
h_height.Draw()
c1.Draw()

In [19]:
h_area.Draw()
c1.Draw()

In [20]:
h_L0Times.Draw()
c1.Draw()

In [21]:
h_L3Times.Draw()
c1.Draw()

In [22]:
h_TimeDiff.Rebin(2)
h_TimeDiff.Draw()
c1.Draw()

Plot the beam on/off files together

In [31]:
fin_beamOn = r.TFile.Open('layerTimes_april29_timeFit_On.root')
fin_beamOff = r.TFile.Open('layerTimes_april26_timeFit_Off.root')

In [32]:
#get all relevant plots
fin_beamOn.ls()

h_PulseTime_on = fin_beamOn.Get('h_pulseTime03')
h_PulseTime_off = fin_beamOff.Get('h_pulseTime03')

h_timeDiff_on = fin_beamOn.Get('h_TimeDiff')
h_timeDiff_off = fin_beamOff.Get('h_TimeDiff')

TFile**		layerTimes_april29_timeFit_On.root	
 TFile*		layerTimes_april29_timeFit_On.root	
  KEY: TH2F	h_pulseTime03;3	Pulse Times Between Layer 0 and 3 [current cycle]
  KEY: TH2F	h_pulseTime03;2	Pulse Times Between Layer 0 and 3 [backup cycle]
  KEY: TH2F	h_pulseTime03;1	Pulse Times Between Layer 0 and 3 [backup cycle]
  KEY: TH1F	h_L0Times;3	Pulse Times Layer 0 [current cycle]
  KEY: TH1F	h_L0Times;2	Pulse Times Layer 0 [backup cycle]
  KEY: TH1F	h_L0Times;1	Pulse Times Layer 0 [backup cycle]
  KEY: TH1F	h_L3Times;3	Pulse Times Layer 3 [current cycle]
  KEY: TH1F	h_L3Times;2	Pulse Times Layer 3 [backup cycle]
  KEY: TH1F	h_L3Times;1	Pulse Times Layer 3 [backup cycle]
  KEY: TH1F	h_TimeDiff;3	Difference in Layer 0 and 3 Times [current cycle]
  KEY: TH1F	h_TimeDiff;2	Difference in Layer 0 and 3 Times [backup cycle]
  KEY: TH1F	h_TimeDiff;1	Difference in Layer 0 and 3 Times [backup cycle]
  KEY: TH1F	h_height;3	Height of Passing Pulses [current cycle]
  KEY: TH1F	h_height;2	Height of Pa

In [33]:
h_PulseTime_on.SetMarkerStyle(7)
h_PulseTime_on.Draw()
h_PulseTime_off.SetMarkerColor(2)
h_PulseTime_off.SetMarkerStyle(7)
h_PulseTime_off.Draw("same")
c1.Draw()

In [35]:
h_timeDiff_off.Rebin(2)
h_timeDiff_on.Rebin(2)
h_timeDiff_off.SetLineColor(2)
h_timeDiff_on.Draw()

h_timeDiff_off.Draw("same")
c1.Draw()